<a href="https://colab.research.google.com/github/Baldezo313/Some-real-world-machine-learning-project/blob/main/PROJET_8_SEGMENTATION_DE_LA_CLIENTELE_D%E2%80%99UNE_ENTREPRISE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROJET 8 : SEGMENTATION DE LA CLIENTELE D’UNE ENTREPRISE

La segmentation de la clientèle d'une entreprise est une pratique de partitionnement des clients
en groupes d'individus de mêmes caractéristiques. Cette stratégie permet de comprendre le
comportement des clients, sur la base de leurs données historiques, et donc d'utiliser de manière
efficace les ressources allouées au Marketing.  

La segmentation se fait par une technique de Machine Learning non supervisé appelée
Clustering. L'algorithme de clustering le plus populaire est KMeans. Dans ce projet, nous
appliquerons cet algorithme pour segmenter la clientèle d'une entreprise.

**LIBRAIRIES FONDAMENTALES**

In [1]:
#Importation des librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Importation des données
customers = pd.read_csv('https://raw.githubusercontent.com/Baldezo313/Some-real-world-machine-learning-project/refs/heads/main/Machine-Learning-par-la-pratique-avec-Python-master/Cust_Segmentation.csv?token=GHSAT0AAAAAAC4VKTEQ2V3NQY27LBX6DSBUZ4H2BJQ')
# Affichage des 5 premières lignes de la dataframe
customers.head()

,Customer Id,Age,Edu,Years Employed,Income,Card Debt,Other Debt,Defaulted,Address,DebtIncomeRatio
0,1,41,2,6,19,0.124,1.073,0.0,NBA001,6.3
1,2,47,1,26,100,4.582,8.218,0.0,NBA021,12.8
2,3,33,2,10,57,6.111,5.802,1.0,NBA013,20.9
3,4,29,2,4,19,0.681,0.516,0.0,NBA009,6.3
4,5,47,1,31,253,9.308,8.908,0.0,NBA008,7.2


In [3]:
# Structure des données
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850 entries, 0 to 849
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Customer Id      850 non-null    int64  
 1   Age              850 non-null    int64  
 2   Edu              850 non-null    int64  
 3   Years Employed   850 non-null    int64  
 4   Income           850 non-null    int64  
 5   Card Debt        850 non-null    float64
 6   Other Debt       850 non-null    float64
 7   Defaulted        700 non-null    float64
 8   Address          850 non-null    object 
 9   DebtIncomeRatio  850 non-null    float64
dtypes: float64(4), int64(5), object(1)
memory usage: 66.5+ KB


Chaque ligne représente un client et chaque colonne représente une variable caractéristique des
clients (Age, Revenu, Adresse, etc.). Toutes les entreprises peuvent disposer facilement de ce
genre de données sur leur clientèle.  

L'algorithme K-means ne traite pas directement les variables qualitatives (comme la plupart des
algorithmes de Machine Learning de Scikit-Learn) puisqu'il est question ici de calcul de
distance. Supprimons donc la colonne Address.

In [4]:
#Suppression de 'Address'
customers = customers.drop("Address", axis = 1)

**STANDARDISATION DES DONNEES**  
Pour donner le même poids à chaque point, nous allons normaliser les données :

In [5]:
# Importation de StandardScaler
from sklearn.preprocessing import StandardScaler


# Standardisation des données
X = customers.values[:, 1:] #Toutes les colonnes sauf la colonne 'Customer Id'

X = np.nan_to_num(X) #Remplacer les Nan par 0 et les infinis par les plus grands nombres finis
cluster_data = StandardScaler().fit_transform(X)
cluster_data

array([[ 0.74291541,  0.31212243, -0.37878978, ..., -0.59048916,
        -0.52379654, -0.57652509],
       [ 1.48949049, -0.76634938,  2.5737211 , ...,  1.51296181,
        -0.52379654,  0.39138677],
       [-0.25251804,  0.31212243,  0.2117124 , ...,  0.80170393,
         1.90913822,  1.59755385],
       ...,
       [-1.24795149,  2.46906604, -1.26454304, ...,  0.03863257,
         1.90913822,  3.45892281],
       [-0.37694723, -0.76634938,  0.50696349, ..., -0.70147601,
        -0.52379654, -1.08281745],
       [ 2.1116364 , -0.76634938,  1.09746566, ...,  0.16463355,
        -0.52379654, -0.2340332 ]])

**EXECUTION DE K-MEANS**  
Faisons un partitionnement en 3 groupes.

In [7]:
# Importation de KMeans
from sklearn.cluster import KMeans

k = 3
k_means = KMeans(init="k-means++", n_clusters=k, n_init=12) #Création d'un objet de laclasse KMeans

k_means.fit(cluster_data) #Application de l'algorithme aux données standardisées

labels = k_means.labels_ #Obtention des étiquettes

#Assignation de chaque client à un groupe (cluster)
customers["Label"] = labels

# Affichage de la dataframe avec les étiquettes
customers.head()

,Customer Id,Age,Edu,Years Employed,Income,Card Debt,Other Debt,Defaulted,DebtIncomeRatio,Label
0,1,41,2,6,19,0.124,1.073,0.0,6.3,2
1,2,47,1,26,100,4.582,8.218,0.0,12.8,0
2,3,33,2,10,57,6.111,5.802,1.0,20.9,1
3,4,29,2,4,19,0.681,0.516,0.0,6.3,2
4,5,47,1,31,253,9.308,8.908,0.0,7.2,0


En faisant la moyenne des variables au niveau de chaque cluster (0, 1 et 2) on aura les
caractéristiques du centroïde (représentant) de chaque groupe.

In [8]:
customers.groupby("Label").mean()

,Customer Id,Age,Edu,Years Employed,Income,Card Debt,Other Debt,Defaulted,DebtIncomeRatio
Label,,,,,,,,,
0,424.408163,43.000000,1.931973,17.197279,101.959184,4.220673,7.954483,0.162393,13.915646
1,424.451807,31.891566,1.861446,3.963855,31.789157,1.576675,2.843355,0.993939,13.994578
2,426.122905,33.817505,1.603352,7.625698,36.143389,0.853128,1.816855,0.000000,7.964991


Nous avons donc le représentant de chacun des groupes de clients ainsi que leurs
caractéristiques. Dans chaque cluster, les clients sont similaires les uns aux autres et on peut
donc considérer comme caractéristiques communes dans chaque cluster, celles de son
représentant c'est-à-dire le centroïde. Ainsi, on peut créer 3 profils de clients :  
* Profil 1 (Cluster 0) : Clients les plus âgés (43 ans en moyenne), ayant les plus gros salaires (102000 dollars par an en moyenne) et légèrement endettés.
* Profil 2 (Cluster 1) : Les plus jeunes (32 ans en moyenne), Salaires les plus bas (environ 32000 dollars par an en moyenne) et très endettés.
* Profil 3 (Cluster 2) : Âges moyens (34 ans), Salaires moyens (environ 36000 dollars par an en moyenne) et financièrement responsables.  
  


## CONCLUSION  
L’implémentation de l’algorithme KMeans dans Python est très simple pour réaliser une
segmentation. La segmentation des clients est très importante pour toute entreprise et permet
d'analyser les comportements des clients. L'algorithme KMeans peut être aussi utilisé dans
d’autres domaines tel que la santé, la recherche scientifique, etc.